In [75]:
import sys
import torch
import torch.nn as nn
from torch.autograd import Variable

torch.manual_seed(777)
#        0   1   2   3   4   5   6
char = ['h','e','l','o','w','r','d']

x_data = [0, 1, 0, 2, 3, 4, 6, 4, 4, 2, 6]
one_hot_lookup = [[1, 0, 0, 0, 0, 0, 0], # 0
                 [0, 1, 0, 0, 0, 0, 0], # 1
                 [0, 0, 1, 0, 0, 0, 0], # 2
                 [0, 0, 0, 1, 0, 0, 0], # 3
                 [0, 0, 0, 0, 1, 0, 0], # 4
                 [0, 0, 0, 0, 0, 1, 0], # 5
                 [0, 0, 0, 0, 0, 0, 1]] # 6

y_data =[0, 1, 2, 2, 3, 4, 4, 3, 5, 2, 6] # hellowworld
x_one_hot = [one_hot_lookup[x] for x in x_data]

inputs = Variable(torch.Tensor(x_one_hot))
labels = Variable(torch.LongTensor(y_data))

num_classes = 7
input_size = 7
hidden_size = 7
batch_size = 1
sequence_length = 1
num_layers = 1

class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.rnn = nn.RNN(input_size=input_size,
                          hidden_size=hidden_size, batch_first=True)

    def forward(self, hidden, x):
        # Reshape input (batch first)
        x = x.view(batch_size, sequence_length, input_size)

        # Propagate input through RNN
        # Input: (batch, seq_len, input_size)
        # hidden: (num_layers * num_directions, batch, hidden_size)
        out, hidden = self.rnn(x, hidden)
        return hidden, out.view(-1, num_classes)

    def init_hidden(self):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size)
        return Variable(torch.zeros(num_layers, batch_size, hidden_size))
    
model = Model()
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr = 0.1)

for epoch in range(50):
    optimizer.zero_grad()
    loss = 0
    hidden = model.init_hidden()
    correct = 0

    sys.stdout.write("predicted string: ")
    for input, label in zip(inputs, labels):
        # print(input.size(), label.size())
      input = input.unsqueeze(0)
      hidden, output = model(hidden, input)
      val, idx = output.max(1)
      sys.stdout.write(char[idx.data[0]])
      loss += criterion(output, label)
      correct += (idx.data[0] == label) # Count correct predictions
    loss.backward()
    optimizer.step()
    accuracy = correct/len(labels)
    print(", epoch: %d, loss: %1.3f, accuracy: %1.3f" % (epoch + 1, loss, accuracy*100)+"%")
    

print("Learning finished!")

Model(
  (rnn): RNN(7, 7, batch_first=True)
)
predicted string: rhrhrrrhrrr, epoch: 1, loss: 22.392, accuracy: 9.091%
predicted string: reelelehlle, epoch: 2, loss: 19.353, accuracy: 27.273%
predicted string: llelowrorle, epoch: 3, loss: 17.058, accuracy: 54.545%
predicted string: llllowrorld, epoch: 4, loss: 15.325, accuracy: 72.727%
predicted string: llllowdorld, epoch: 5, loss: 13.958, accuracy: 72.727%
predicted string: llllowdorld, epoch: 6, loss: 12.960, accuracy: 72.727%
predicted string: llllowworld, epoch: 7, loss: 12.088, accuracy: 81.818%
predicted string: lellowworld, epoch: 8, loss: 11.335, accuracy: 90.909%
predicted string: lellowworld, epoch: 9, loss: 10.687, accuracy: 90.909%
predicted string: lellowworld, epoch: 10, loss: 10.119, accuracy: 90.909%
predicted string: lellowworld, epoch: 11, loss: 9.732, accuracy: 90.909%
predicted string: lellowworld, epoch: 12, loss: 9.421, accuracy: 90.909%
predicted string: lellowworld, epoch: 13, loss: 9.137, accuracy: 90.909%
predi